In [ ]:
#————————————————————

# Name: Data Cleansing and Dedulpication of Openfoodfacts (V1)

# Purpose:


# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

# General Sources:

# Additionals:

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [9]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
import numpy as np
from pathlib import Path
import pandas as pd
import requests
import random
import jsonlines

In [2]:
# Import recipes csv

path_input = r"C:\Python\openai-lab\data\recipes.csv" #Change path if required
df = pd.read_csv(path_input , sep=',', on_bad_lines='skip', low_memory=False)

In [3]:
# Remove columns

df = df.drop(['id', 'contributor_id', 'submitted'], axis=1)

# Remove double whitespaces from name

df["name"] = df["name"].str.replace(r'\s+', ' ', regex=True)

# Create subset of data with only vegan recipes
df = df[df['tags'].str.contains("vegan")]

# Create subset of data
df = df.sample(frac = 0.015)

# Creating subset of the subset with 70% values of original dataframe
df_recipe_training_set = df.sample(frac = 0.7)

#Creating dataframe with rest of the 30% values
df_recipe_validation_set = df.drop(df_recipe_training_set.index)

In [66]:
#df.iloc[2:3].to_json(orient='records', lines=True).replace('"\n"', '')

'{"name":"fast fix gazpacho","minutes":10,"tags":"[\'15-minutes-or-less\', \'time-to-make\', \'course\', \'cuisine\', \'preparation\', \'low-protein\', \'healthy\', \'soups-stews\', \'spanish\', \'european\', \'no-cook\', \'low-fat\', \'vegan\', \'vegetarian\', \'food-processor-blender\', \'dietary\', \'low-cholesterol\', \'low-saturated-fat\', \'low-calorie\', \'low-carb\', \'healthy-2\', \'low-in-something\', \'equipment\', \'small-appliance\', \'presentation\', \'served-cold\', \'technique\']","nutrition":"[75.6, 0.0, 45.0, 32.0, 5.0, 0.0, 5.0]","n_steps":6,"steps":"[\'coarsely chop cucumber , pepper & onion\', \'add half of the veggies to a food processor along with the garlic , 1 cup v8 juice , lime juice , hot sauce & salt\', \'puree till smooth\', \'pour into a bowl & add remaining 3 cups v8\', \'chill\', \'serve cold with remaining chopped veggies and cilantro as garnish\']","description":"for some reason, my husband thinks he loves gazpacho.  yet if i make a regular size batch

In [70]:
#re.sub(':', '', test)

'{"name""fast fix gazpacho","minutes"10,"tags""[\'15-minutes-or-less\', \'time-to-make\', \'course\', \'cuisine\', \'preparation\', \'low-protein\', \'healthy\', \'soups-stews\', \'spanish\', \'european\', \'no-cook\', \'low-fat\', \'vegan\', \'vegetarian\', \'food-processor-blender\', \'dietary\', \'low-cholesterol\', \'low-saturated-fat\', \'low-calorie\', \'low-carb\', \'healthy-2\', \'low-in-something\', \'equipment\', \'small-appliance\', \'presentation\', \'served-cold\', \'technique\']","nutrition""[75.6, 0.0, 45.0, 32.0, 5.0, 0.0, 5.0]","n_steps"6,"steps""[\'coarsely chop cucumber , pepper & onion\', \'add half of the veggies to a food processor along with the garlic , 1 cup v8 juice , lime juice , hot sauce & salt\', \'puree till smooth\', \'pour into a bowl & add remaining 3 cups v8\', \'chill\', \'serve cold with remaining chopped veggies and cilantro as garnish\']","description""for some reason, my husband thinks he loves gazpacho.  yet if i make a regular size batch, he ea

In [4]:
training_data = []
validation_data = []

system_message = "Act as a head chef and create a flavourful recipe from a list of ingredients"

def create_user_message(row):
    stripped = row.ingredients.apply(lambda x: str(x).strip("[]").replace("'", ""))
    return f"""{stripped.values[0]}"""

def create_assistant_message(row):
    json_record = row.to_json(orient='records', lines=True)
    return f"""{json_record}"""

def create_final_message(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    assistant_message = create_assistant_message(row)
    messages.append({"role": "assistant", "content": assistant_message})

    return {"messages": messages}

create_final_message(df.iloc[:3])

{'messages': [{'role': 'system',
   'content': 'Act as a head chef and create a flavourful recipe from a list of ingredients'},
  {'role': 'user',
   'content': 'coconut oil, powdered sugar, vanilla, coconut milk, cherry preserves'},
  {'role': 'assistant',
   'content': '{"name":"natural non dairy pink frosting","minutes":10,"tags":"[\'lactose\', \'15-minutes-or-less\', \'time-to-make\', \'course\', \'preparation\', \'occasion\', \'low-protein\', \'5-ingredients-or-less\', \'desserts\', \'easy\', \'kosher\', \'vegan\', \'vegetarian\', \'cakes\', \'dietary\', \'low-sodium\', \'cake-fillings-and-frostings\', \'free-of-something\', \'low-in-something\', \'taste-mood\', \'sweet\', \'equipment\', \'small-appliance\', \'mixer\']","nutrition":"[2397.3, 188.0, 1277.0, 2.0, 4.0, 529.0, 111.0]","n_steps":5,"steps":"[\'cream coconut oil until light\', \'add vanilla and sugar\', \'mix well\', \'add coconut milk and preserves\', \'beat on high until creamy\']","description":"this is a quick, delic

In [5]:
for i in range(1,len(df_recipe_training_set) + 1,1):
    # change a = a.append(i) to
    output = create_final_message(df_recipe_training_set.iloc[i-1:i])   
    training_data.append(output)

In [6]:
for i in range(1,len(df_recipe_validation_set) + 1,1):
    # change a = a.append(i) to
    output = create_final_message(df_recipe_validation_set.iloc[i-1:i])   
    validation_data.append(output)

In [7]:
for example in training_data[104:105]:
    print(example)

{'messages': [{'role': 'system', 'content': 'Act as a head chef and create a flavourful recipe from a list of ingredients'}, {'role': 'user', 'content': 'potatoes, chili beans, chili powder, salt, oil, cheese, sour cream'}, {'role': 'assistant', 'content': '{"name":"bean stuffed potatoes","minutes":85,"tags":"[\'time-to-make\', \'course\', \'main-ingredient\', \'preparation\', \'occasion\', \'low-protein\', \'healthy\', \'lunch\', \'side-dishes\', \'potatoes\', \'vegetables\', \'low-fat\', \'vegan\', \'vegetarian\', \'dietary\', \'low-sodium\', \'low-cholesterol\', \'low-saturated-fat\', \'low-calorie\', \'healthy-2\', \'low-in-something\', \'brunch\', \'4-hours-or-less\']","nutrition":"[211.7, 2.0, 7.0, 9.0, 13.0, 1.0, 14.0]","n_steps":13,"steps":"[\'wash potatoes\', \'prick several times with a fork\', \'bake in the pre heated oven at 180c \\/ 375f for 45-60 minutes until potatoes are tender\', \'let cool a bit\', \'cut off a slice of the long side of the potatoes\', \'scrape out fle

In [8]:

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)



In [10]:
write_jsonl(training_data, "recipes-training-set.jsonl")
write_jsonl(training_data, "recipes-validation-set.jsonl")

In [5]:
#path_output = r"C:\Python\openai-lab\data\openfoodfacts.csv"
path_output = r"C:\Python\openai-lab\data\recipes_short.csv" #Change path if required

#df_products.to_excel(path_output, sheet_name='Sheet1', index = False)
df.to_csv(path_output, sep='\t', encoding='utf-8', index=False)
